# RNN 이용한 텍스트 생성

In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size = len(tokenizer.word_index)+1
print("단어 집합의 크기 : ", vocab_size)

단어 집합의 크기 :  12


In [10]:
print(tokenizer.word_index)


{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


훈련 데이터 만들기 

In [13]:
sequences = list()
for line in text.split('\n'):
    encoded= tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
print("학습에 사용할 샘플의 개수 : ", len(sequences))
    

학습에 사용할 샘플의 개수 :  11


In [15]:
max_len = max(len(l) for l in sequences)
print("샘플의 최대 길이 :", max_len)

샘플의 최대 길이 : 6


In [17]:
# 길이가 6보다 짧은 모든 샘플에 대하여 앞에 0을 채워주는 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [18]:
# 레이블링
sequences = np.array(sequences)
X=sequences[:, :-1]
y= sequences[:,-1]

In [19]:
print(X)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]


In [20]:
print(y)

[ 3  1  4  5  1  7  1  9 10  1 11]


In [21]:
# 원핫 인코딩
y= to_categorical(y, num_classes=vocab_size)

In [22]:
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


RNN 모델 설계하기

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

model = Sequential()
model.add(Embedding(vocab_size, 10))
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X,y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 2s - loss: 2.4804 - accuracy: 0.0000e+00 - 2s/epoch - 2s/step
Epoch 2/200
1/1 - 0s - loss: 2.4669 - accuracy: 0.0909 - 27ms/epoch - 27ms/step
Epoch 3/200
1/1 - 0s - loss: 2.4535 - accuracy: 0.1818 - 25ms/epoch - 25ms/step
Epoch 4/200
1/1 - 0s - loss: 2.4400 - accuracy: 0.2727 - 16ms/epoch - 16ms/step
Epoch 5/200
1/1 - 0s - loss: 2.4265 - accuracy: 0.3636 - 14ms/epoch - 14ms/step
Epoch 6/200
1/1 - 0s - loss: 2.4129 - accuracy: 0.3636 - 14ms/epoch - 14ms/step
Epoch 7/200
1/1 - 0s - loss: 2.3990 - accuracy: 0.4545 - 15ms/epoch - 15ms/step
Epoch 8/200
1/1 - 0s - loss: 2.3849 - accuracy: 0.4545 - 15ms/epoch - 15ms/step
Epoch 9/200
1/1 - 0s - loss: 2.3704 - accuracy: 0.4545 - 16ms/epoch - 16ms/step
Epoch 10/200
1/1 - 0s - loss: 2.3555 - accuracy: 0.4545 - 15ms/epoch - 15ms/step
Epoch 11/200
1/1 - 0s - loss: 2.3400 - accuracy: 0.5455 - 15ms/epoch - 15ms/step
Epoch 12/200
1/1 - 0s - loss: 2.3240 - accuracy: 0.5455 - 14ms/epoch - 14ms/step
Epoch 13/200
1/1 - 0s - loss: 2.3074 

In [24]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복 횟수
    init_word = current_word
    sentence= ''
    
    # n 번 반복
    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result 에 저장
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)
        
        for word, index in tokenizer.word_index.items():
            if index==result:
                break
        # 현재 단어 + ' ' + 예측단어를 현재 단어로 변경
        current_word = current_word + ' ' + word
        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word
        
    sentence = init_word + sentence
    return sentence

In [25]:
print(sentence_generation(model, tokenizer, '경마장에', 4))

경마장에 있는 말이 뛰고 있다


In [26]:
print(sentence_generation(model, tokenizer, '그의', 4))

그의 말이 법이다 오는 말이


In [27]:
print(sentence_generation(model, tokenizer, '가는', 5))


가는 말이 고와야 오는 말이 곱다
